In [147]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

def RMSEvec(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        if ground[i] != 0:
            error += (ground[i] - predict[i])**2
            n += 1
                
    return np.sqrt(error/n)

def meanError(ground_truth,new_res):
    return np.mean(abs((new_res - ground_truth)[ground_truth!=0]))

In [149]:
def dictfromR(R):

    R_dict = {"Users": np.empty([0]), "Movies": np.empty([0]), "Ratings": np.empty([0])}

    for i in range(len(R)):
        for j in range(len(R[0])):
            if R[i,j] != 0:
                R_dict["Users"] = np.append(R_dict["Users"],i)
                R_dict["Movies"] = np.append(R_dict["Movies"],j)
                R_dict["Ratings"] = np.append(R_dict["Ratings"],R[i,j])

    return R_dict

In [150]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/DATA/MVA/Graphes/Projet/cf-cold-start/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.]), 'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5]), 'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.])}


In [151]:
num_cold = 5
cold_movies = np.argsort([np.sum(R[i,:] for i in range(len(R)))][0])[-num_cold:]

In [152]:
to_keep = 5
ground_truth = []
sel = []
for i,c in enumerate(cold_movies):
    sel.append(np.where(R[:,c] != 0)[0])
    np.random.shuffle(sel[i])
    sel[i] = sel[i][:len(sel[i])-to_keep]
    ground_truth.append(copy.deepcopy(R[:,c]))
    R[sel[i],c]=0
    

In [153]:
R_dictCopy = copy.deepcopy(R_dict)
R_dict = dictfromR(R)

In [154]:
np.where(R_dict['Movies'] == 321)

(array([ 7284, 37687, 54490, 58837, 94323]),)

In [155]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [156]:
R_rec = np.dot(als.U,np.transpose(als.V))

In [157]:
print(RMSE(R,R_rec))
print(RMSE(R,(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))))

0.702271185299
0.950078183622


In [158]:
for i in range(num_cold):
    print(RMSEvec(ground_truth[i], R_rec[:,321]))

0.989074750541
1.07275264958
1.22907273624
0.964968658164
1.07876016755


In [159]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [160]:
np.max(R_rec)

8.5118980224953358

In [161]:
lp = LaplacianParams()

# sim = similarity(als.U)
sim = build_graph(als.U, GraphParams())
# Seems to work better with U... 

print(sim)

[[ 0.          0.          0.         ...,  0.          0.56677973  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.78440386]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [162]:
L = build_laplacian(sim,lp)

print(L.shape)

(671, 671)


In [163]:
supp = 100
test_vec = copy.deepcopy(R[:,321])*2
# test_vec[:supp] = [0 for i in range(supp)]
test_vec.shape

(671,)

In [164]:
# test_vec

In [165]:
hfs0, confidence = simple_hfs(als.U, test_vec, L, sim)
# hfs0/2

/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [166]:
maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])

In [167]:
lim = np.percentile(maxconfidences, 1)
for i in range(num_cold):
#     print(RMSEvec(ground_truth[i]*(maxconfidences > lim),hfs0/2))
    print(RMSEvec(ground_truth[i]*(maxconfidences > lim), R_rec[:,cold_movies[i]]))

0.833829513812
0.912802821767
0.781259282713
0.966361740479
0.697755164101


In [168]:
for i in range(num_cold):
#     print(meanError(ground_truth[i],hfs0/2))
    print(meanError(ground_truth[i],R_rec[:,cold_movies[i]]))

0.632456550732
0.694384313159
0.575604803647
0.742706113125
0.531258731055


In [169]:
# elmnt = 321
# val = []
# print(RMSEvec(R[:,elmnt],R_rec[:,elmnt]))
# for supp in range(1,671,10):
#     test_vec = copy.deepcopy(R[:,elmnt])*2
#     test_vec[:supp] = [0 for i in range(supp)]

#     hfs0 = simple_hfs(als.U, test_vec, L, sim)
#     val.append(RMSEvec(R[:,elmnt],hfs0/2))
    
# plt.plot(range(1,671,10),val)

In [170]:
lhfs = []
lconf = []
for i in range(len(R[0])):
    if i%1000 == 0:
        print(i)
    hfs0, confidence = simple_hfs(als.U, R[:,i]*2, L, sim)
    maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])
    
    lim = np.percentile(maxconfidences, 95)
    
    lhfs.append(hfs0/2)
    lconf.append(maxconfidences > lim)

R_barre = np.vstack(lhfs).T
confs = np.vstack(lconf).T
    
# R_barre[R_barre < 1] = .5
# R_barre[R_barre > 5] = 5

0


/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


1000
2000
3000
4000
5000
6000
7000
8000
9000


In [171]:
R_barre_limited = R_barre * confs

In [172]:
sum(R_barre_limited)

array([   0.,    0.,    0., ...,  170.,  102.,  170.])

In [173]:
print(RMSE(R_barre_limited,R_rec))

0.502225370332


In [174]:
R_barre_final = copy.deepcopy(R_barre_limited)
R_barre_final[R != 0] = 0

In [175]:
# R_dict_barre = dictfromR(R_barre_final)

In [176]:
N = len(R)
M = len(R[0])
K = 4

als_trans = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")

ans = als_trans.fitTransductive(R_dict,R_barre_final,C1=1,C2=0.1)

Als created


In [177]:
R_rec_trans = np.dot(als_trans.U,np.transpose(als_trans.V))
print(RMSE(R_rec_trans,R_rec))

0.339500639286


In [178]:
for i in range(num_cold):
    print("movie "+str(i+1))
    print(RMSEvec(ground_truth[i],R_rec_trans[:,cold_movies[i]]))
    print(RMSEvec(ground_truth[i],R_rec[:,cold_movies[i]]))
    print(RMSEvec(R_barre_limited[:,cold_movies[i]]*(ground_truth[i]>0),ground_truth[i]))

movie 1
0.874056944757
0.832395598607
0.612372435696
movie 2
0.909559328801
0.912802821767
0.632455532034
movie 3
0.807238304348
0.78058073159
0.767649473579
movie 4
0.929033138521
0.964968658164
0.623609564462
movie 5
0.698069166614
0.711507992296
0.540061724867


0.930864602802
0.97011494657
1.00036650175 (90 1 0.1)

0.910201784408
0.97011494657
1.00036650175 (90 1 0.5)

0.931048133417
0.97011494657
1.00036650175 (95 1 0.5)

In [179]:
R_barre

array([[ 4. ,  3. ,  3. , ...,  5. ,  3. ,  5. ],
       [ 4. ,  4. ,  4. , ...,  5. ,  3. ,  5. ],
       [ 4. ,  2.5,  3.5, ...,  5. ,  3. ,  5. ],
       ..., 
       [ 4. ,  2.5,  3.5, ...,  5. ,  3. ,  5. ],
       [ 4. ,  3. ,  4. , ...,  5. ,  3. ,  5. ],
       [ 5. ,  3. ,  4. , ...,  5. ,  3. ,  5. ]])